In [ ]:
import torch
import math

In [ ]:
theta = torch.rand(100000)*2*math.pi
x = torch.rand(100000)*100
y = torch.rand(100000)*100

alpha = torch.rand(100)
beta = torch.rand(100)
gamma = torch.rand(100)

In [ ]:
def se2_matrix(bx, by, btheta):   
    bcos = torch.cos(btheta)
    bsin = torch.sin(btheta)
    
    bGg = torch.zeros((bx.shape[0], 3, 3), device=None)
    bGg[:,0,0] = bcos
    bGg[:,0,1] = -bsin
    bGg[:,0,2] = bx
    bGg[:,1,0] = bsin
    bGg[:,1,1] = bcos
    bGg[:,1,2] = by
    bGg[:,2,2] = 1.
    
    return bGg

def so3_matrix(balpha, bbeta, bgamma):
    cosa = torch.cos(alpha)
    sina = torch.sin(alpha)    
    Ra = torch.zeros((alpha.shape[0], 3, 3))
    Ra[:,1,1] = cosa
    Ra[:,1,2] = -sina
    Ra[:,2,1] = sina
    Ra[:,2,2] = cosa
    Ra[:,0,0] = 1.

    cosb = torch.cos(beta)
    sinb = torch.sin(beta)   
    Rb = torch.zeros((beta.shape[0], 3, 3))
    Rb[:,0,0] = cosb
    Rb[:,0,2] = sinb
    Rb[:,2,0] = -sinb
    Rb[:,2,2] = cosb
    Rb[:,1,1] = 1.

    cosc = torch.cos(gamma)
    sinc = torch.sin(gamma) 
    Rc = torch.zeros((gamma.shape[0], 3, 3))
    Rc[:,0,0] = cosc
    Rc[:,0,1] = -sinc
    Rc[:,1,0] = sinc
    Rc[:,1,1] = cosc
    Rc[:,2,2] = 1.
    
    return Rc @ Rb @ Ra

In [ ]:
se2_matrix(x, y, theta)

In [ ]:
%timeit so3_matrix(alpha, beta, gamma)

In [ ]:
bGg = se2_matrix(x, y, theta)

In [ ]:
(torch.inverse(bGg[:100]).unsqueeze(1) @ torch.inverse(bGg).unsqueeze(0))

In [ ]:
torch.inverse(bGg)

In [ ]:
torch.complex(torch.cos(theta), torch.sin(theta))

In [ ]:
torch.log(torch.complex(torch.cos(theta), torch.sin(theta)))

In [ ]:
torch.sign(torch.sin(theta))*torch.acos(torch.cos(theta))

In [ ]:
def se2_log(bbGg):
    # theta = torch.sign(bGg[...,1,0]) * torch.acos(bGg[...,0,0]) % math.pi
    # x = bGg[...,0,2]
    # y = bGg[...,1,2]
    # 
    # mask = (theta == 0)
    # 
    # c1 = 0.5 * theta * (y + x * torch.cos(theta/2)/torch.sin(theta/2))
    # c1[mask] = x[mask]
    # 
    # c2 = 0.5 * theta * (-x + y * torch.cos(theta/2)/torch.sin(theta/2))
    # c2[mask] = y[mask]
# 
    # c3 = theta
    #     
    # return torch.stack((c1, c2, c3), dim=-1)

    theta = (bbGg[3].sign() * bbGg[0].acos()).mod(math.pi, -math.pi/2)
    x = bbGg[2]
    y = bbGg[5]
    
    c1 = theta/2 * y + x * (theta/2).cos()*(theta/2).xdivsinx()
    c2 = -theta/2 * x + y * (theta/2).cos()*(theta/2).xdivsinx()
    c3 = theta
    
    return LazyTensor.cat((c1, c2, c3), dim=-1)

In [ ]:
x = torch.zeros(100)
y = torch.zeros(100)
theta = torch.rand(100)

In [ ]:
bGg = se2_matrix(x,y,theta)
bGg.shape

In [ ]:
se2_log(torch.inverse(bGg).unsqueeze(1) @ bGg.unsqueeze(0))

In [ ]:
xi = Vi(bGg)
xj = Vj(bGg)

In [ ]:
LazyTensor.keops_tensordot(xi, xj, (3, 3), (3, 3), (1,), (0,)).sum_reduction(dim=-1)

In [ ]:
from pykeops.torch import LazyTensor, Pm, Vi, Vj
import torch
import math

S = Pm(torch.tensor([1., 1., 1.]))

x = torch.zeros(4)
y = torch.zeros(4)
theta = torch.tensor([0., math.pi/4, math.pi/2, 3/4 * math.pi])

Gg = se2_matrix(x, y, theta)
Gh = torch.inverse(Gg)

xi = Vi(Gh.reshape(4, -1))  # sources
xj = Vj(Gg.reshape(4, -1))  # targets

xixj = LazyTensor.keops_tensordot(xi, xj, (3, 3), (3, 3), (1,), (0,)) # g^-1 * h

se2_log(xixj).weightedsqnorm(S).Kmin_argKmin(4, dim=1)

In [ ]:
(0. - math.pi/4)**2